In [1]:
'''
ABC - Time Series Model from Historical Data
'''

'\nABC - Time Series Model from Historical Data\n'

In [4]:
#from pmdarima.arima import auto_arima
import pandas as pd
import numpy as np
from datetime import datetime 
import sys
sys.path.append('../')

In [41]:
def solution():
    result=[]
    #Reading the datasets
    historical_Data=pd.read_csv('res/Historical_Data.csv')
    #print(historical_Data.head())
    #print(historical_Data.columns)
    #print(historical_Data.shape)
    #print(historical_Data.dtypes)
    #print(historical_Data.isnull().sum())
    #print(historical_Data.describe())
    
    #Converting Date column into Datetime
    historical_Data['to_date'] = pd.to_datetime(historical_Data['Date'],format="%Y%m%d")
    print(historical_Data.head())
    
    
    day_high = historical_Data.groupby(['Date'])['Sold_Units'].sum()
    day_high = day_high.reset_index(name='Sold_Units')

    #Total no of days where unot sold > 3
    total_sold_3 = day_high[day_high.Sold_Units > 3]['Sold_Units'].count()
    result.append(total_sold_3)
    
    #Sales of country FR in the month of August
    historical_Data['month'] = pd.DatetimeIndex(historical_Data['to_date']).month
    total_sold_FR = historical_Data[ (historical_Data['Country_Code'] == 'FR') 
                                    & (historical_Data['month'] == 8) ]['Sold_Units'].sum()
    result.append(total_sold_FR)
    

    #Total Number of Units sold in AT
    total_sold_AT = historical_Data[historical_Data.Country_Code == 'AT']['Sold_Units'].sum()
    result.append(total_sold_AT)
    
    
    print(result)
    result=pd.DataFrame(result)
    #writing output to output.csv
    result.to_csv('output/output.csv', header=False, index=False)
    
solution()

       Date  Article_ID Country_Code  Sold_Units
0  20170817        1132           AT           1
1  20170818        1132           AT           1
2  20170821        1132           AT           1
3  20170822        1132           AT           1
4  20170906        1132           AT           1
       Date  Article_ID Country_Code  Sold_Units    to_date
0  20170817        1132           AT           1 2017-08-17
1  20170818        1132           AT           1 2017-08-18
2  20170821        1132           AT           1 2017-08-21
3  20170822        1132           AT           1 2017-08-22
4  20170906        1132           AT           1 2017-09-06
[725, 121, 1330]


In [43]:
from pmdarima import auto_arima
import pandas as pd
import numpy as np
import datetime

In [135]:
def solution():
    #Reading the datasets
    historical_Data=pd.read_csv('res/Historical_Data.csv')
    
    #Converting Date column into Datetime
    historical_Data['to_date'] = pd.to_datetime(historical_Data['Date'],format="%Y%m%d")
    historical_Data.drop(columns='Date',inplace=True)
    historical_Data.rename(columns = {'to_date':'Date'},inplace=True)
    
    #min_date = historical_Data['Date'].min()
    #max_date = historical_Data['Date'].max()
    #print(min_date,max_date)
    
    #Creating Date as Index Column
    #historical_Data.set_index('Date',inplace = True)
    #print(historical_Data.head())
    
    #creating dummy dataframe
    column_names = historical_Data.columns
    df = pd.DataFrame(columns = column_names)
    
    #Reading dataframe by country code
    for c in historical_Data['Country_Code'].unique():
        df_temp = historical_Data[ historical_Data['Country_Code'] == c ]
        
        #finding Min and max of date for a country code
        min_date = df_temp['Date'].min()
        max_date = df_temp['Date'].max()
        
        #creating a date range for country code
        idx = pd.date_range(min_date, max_date, freq='D')
        #print(len(idx))
        
        #creating dataframe for date range
        date_list = pd.DataFrame(idx,columns = ['Date'])
        #print(date_list)
        #print(df_temp.shape,date_list.shape)
        
        #merging two dataframes on Date column
        df_date = pd.merge(df_temp, date_list, on='Date', how='right')
        
        #Filling NAN columns
        df_date['Country_Code'] = df_date['Country_Code'].fillna(c)
        df_date['Article_ID'] = df_date['Article_ID'].fillna(0)
        df_date['Sold_Units'] = df_date['Sold_Units'].fillna(0)
        
        #print(df_date.shape)
        
        #Concating two dataframes
        df = pd.concat([df,df_date])
        #print(df_date)
        #print("-----------------------------------------------")

    #Finding start date of sale
    FR_start_sell = df['Date'][ (df['Country_Code'] == 'FR') & (df['Sold_Units'] > 0 ) ].min()
    print(FR_start_sell.date())
    
    #Finding all dates when there was no sale
    AT_non_sell = df['Date'][ (df['Country_Code'] == 'AT') & (df['Sold_Units'] == 0 ) ].unique().size
    print(AT_non_sell)
    
    result=[FR_start_sell.date(),AT_non_sell]
    result=pd.DataFrame(result)
    #writing output to output.csv
    result.to_csv('output/output.csv', header=False, index=False)
    

solution()

2017-01-02
237


In [137]:
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import datetime

In [190]:
def solution():
    #to store results
    result = []
    #Reading the datasets
    historical_Data=pd.read_csv('res/Historical_Data.csv')
    
    #Converting Date column into Datetime
    historical_Data['Date'] = pd.to_datetime(historical_Data['Date'],format="%Y%m%d")
    
    #making date as index column
    historical_Data.set_index('Date', inplace=True)
    #sorting the dataframe based on index
    historical_Data = historical_Data.sort_index()
    
    
    #Running model for each country
    for c in historical_Data['Country_Code'].unique():
        
        #Extracting Data based on country code for all sold units
        df_temp = historical_Data[ (historical_Data['Country_Code'] == c ) & (historical_Data['Sold_Units'] > 0) ]['Sold_Units']
        #df_temp.set_index('Date', inplace=True)
        #print(df_temp.head())
        #print(df_temp.shape)
        
        
        #Splitting the data into test and train
        test = df_temp.iloc[-10:]
        train = df_temp.iloc[:(df_temp.shape[0] -10)]
        #print(df_temp.iloc[:-10].shape)
        #print(test.shape,train.shape)
        
        #Using Auto-ARIMA model
        
        model = auto_arima(train, trace=True, supress_warnings=True, error_action='ignore')
        model.fit(train)
        #checking model summary
        #print(model.summary())
        
        #predicting the data
        
        prediction = pd.DataFrame(model.predict(n_periods = 10),index = test.index)

        prediction.columns = ['Predicted_units']
        
        #print(prediction)
        
        Final_df = pd.merge(test, prediction, left_index=True, right_index=True)
        print(Final_df)
        mse = mean_absolute_error(Final_df['Sold_Units'], Final_df['Predicted_units'])
        #print(mse)
        
        #print(round(mse,3))
        #Appending mse to result dataframe
        result.append(round(mse,3))
        
        #break
        #print("-----------------------------------------------")
    
    
    print(result)
    result=pd.DataFrame(result)
    #writing output to output.csv
    result.to_csv('output/output.csv', header=False, index=False)
    
solution()    

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=2427.667, Time=0.96 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=2428.794, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=2424.411, Time=0.04 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=2424.539, Time=0.05 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=3291.023, Time=0.01 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=2426.345, Time=0.09 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=2424.858, Time=0.62 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=2426.190, Time=0.72 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=2765.406, Time=0.02 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 2.536 seconds
            Sold_Units  Predicted_units
Date                                   
2019-02-09           2         1.735010
2019-02-10           5         1.710860
2019-02-10           5         1.708659
2019-02-10           2         1.710860
2019-02-10           2         1.708659
2019-0

 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=8496.349, Time=0.72 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=8468.827, Time=2.43 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=8485.879, Time=1.20 sec
 ARIMA(4,1,3)(0,0,0)[0] intercept   : AIC=8456.888, Time=1.94 sec
 ARIMA(5,1,3)(0,0,0)[0] intercept   : AIC=8458.887, Time=1.97 sec
 ARIMA(4,1,4)(0,0,0)[0] intercept   : AIC=8459.088, Time=2.40 sec
 ARIMA(3,1,4)(0,0,0)[0] intercept   : AIC=8465.066, Time=1.91 sec
 ARIMA(5,1,2)(0,0,0)[0] intercept   : AIC=8488.621, Time=1.77 sec
 ARIMA(5,1,4)(0,0,0)[0] intercept   : AIC=8460.647, Time=1.94 sec
 ARIMA(4,1,3)(0,0,0)[0]             : AIC=8454.896, Time=0.80 sec
 ARIMA(3,1,3)(0,0,0)[0]             : AIC=8466.831, Time=1.21 sec
 ARIMA(4,1,2)(0,0,0)[0]             : AIC=8485.331, Time=0.89 sec
 ARIMA(5,1,3)(0,0,0)[0]             : AIC=8456.895, Time=1.13 sec
 ARIMA(4,1,4)(0,0,0)[0]             : AIC=8456.904, Time=1.23 sec
 ARIMA(3,1,2)(0,0,0)[0]             : AIC=8489.720, Time=0.35 sec
 ARIMA(3,1

In [209]:
def solution():
    #to store results
    result = []
    #Reading the datasets
    historical_Data=pd.read_csv('res/Historical_Data.csv')
    
    #Converting Date column into Datetime
    historical_Data['Date'] = pd.to_datetime(historical_Data['Date'],format="%Y%m%d")
    
    #making date as index column
    #historical_Data.set_index('Date', inplace=True)
    #sorting the dataframe based on index
    #historical_Data = historical_Data.sort_index()
    
    
    #Running model for each country
    for c in historical_Data['Country_Code'].unique():
        
        #Extracting Data based on country code for all sold units
        df_temp = historical_Data[ (historical_Data['Country_Code'] == c ) & (historical_Data['Sold_Units'] > 0) ][['Date','Sold_Units']]
        #df_temp.set_index('Date', inplace=True)
        #print(df_temp.head())
        #print(df_temp.shape)
        
        df_temp = pd.DataFrame(df_temp.groupby(['Date'])['Sold_Units'].sum())
        #print(df_temp.columns) # = ['Sold_Units']
        df_temp = df_temp.sort_index()
        #print(df_temp)
        #print(df_temp.shape)
        
        
        #Splitting the data into test and train
        test = df_temp.iloc[-10:]
        train = df_temp.iloc[:(df_temp.shape[0] -10)]
        #print(df_temp.iloc[:-10].shape)
        #print(test.shape,train.shape)
        
        #Using Auto-ARIMA model
        
        model = auto_arima(train, trace=True, supress_warnings=True, error_action='ignore')
        model.fit(train)
        #checking model summary
        #print(model.summary())
        
        #predicting the data
        
        prediction = pd.DataFrame(model.predict(n_periods = 10),index = test.index)

        prediction.columns = ['Predicted_units']
        
        #print(prediction)
        
        Final_df = pd.merge(test, prediction, left_index=True, right_index=True)
        print(Final_df)
        mse = mean_absolute_error(Final_df['Sold_Units'], Final_df['Predicted_units'])
        #print(mse)
        
        #print(round(mse,3))
        #Appending mse to result dataframe
        result.append(round(mse,3))
        
        #break
        #print("-----------------------------------------------")
    
    
    print(result)
    result=pd.DataFrame(result)
    #writing output to output.csv
    result.to_csv('output/output.csv', header=False, index=False)
    
solution()  

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=2119.693, Time=0.44 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2439.681, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2270.477, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2114.340, Time=0.09 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2437.689, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2116.011, Time=0.21 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=2116.065, Time=0.15 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=2118.338, Time=0.17 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=2113.193, Time=0.04 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2114.951, Time=0.05 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=2114.989, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=2268.494, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=2117.192, Time=0.05 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0]          
Total fit time: 1.341 seconds
            Sold_Units